<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Day13_CustomerSupport_SaleBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Day 13 — Customer Support: SaleBot new clients

## 🎓 Теория

### LlamaIndex: `BM25Retriever`

BM25Retriever реализует алгоритм BM25 (Best Matching 25) — улучшенную версию TF-IDF, которая учитывает:

- Частоту термина (TF): сколько раз слово встречается в документе.

- Обратную частоту документа (IDF): насколько редко слово встречается в корпусе.

- Длину документа: нормализация для избежания смещения в сторону длинных
  документов.


Это делает BM25 эффективным для поиска по ключевым словам, особенно в случаях, когда требуется точное совпадение терминов.

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever

### Загрузка документов
documents = SimpleDirectoryReader("faq_data").load_data()

### Разделение документов на узлы
splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)

### Инициализация BM25Retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=3  # Количество возвращаемых результатов
)

### Выполнение запроса
query = "Как получить подписку?"
retrieved_nodes = bm25_retriever.retrieve(query)

# Вывод результатов
for node in retrieved_nodes:
    print(node.text)

### LangGraph: Task Queues

Очереди задач в LangGraph позволяют организовать циклическую обработку состояний.

```python
from langgraph.graph import StateGraph

graph = StateGraph()

@graph.node()
def detect_user(state):
    if state["user_type"] == "new":
        state["tasks"] = ["send_welcome", "faq_reply"]
    else:
        state["tasks"] = ["handover_to_manager"]
    return state

@graph.node()
def task_worker(state):
    task = state["tasks"].pop(0)
    if task == "send_welcome":
        print("👋 Добро пожаловать! Чем могу помочь?")
    elif task == "faq_reply":
        answer = query_engine.query(state["user_question"])
        print(f"🤖 {answer.response}")
    elif task == "handover_to_manager":
        print("🔄 Перевожу вас к менеджеру.")
    return state

graph.set_entry_point(detect_user)
graph.add_edge(detect_user, task_worker)
graph.set_finish_point(task_worker)

compiled = graph.compile()
compiled.invoke({
    "user_type": "new",
    "user_question": "Как получить подписку?"
})
```

---

### n8n: автоматическая обработка новых клиентов

1. Приветствие нового клиента
2. Ответ на FAQ через OpenAI
3. Лог в Google Sheets
4. Фидбек (оценка ответа)
5. Автоэскалация, если оценка плохая


In [1]:
!pip install -q llama-index llama-index-retrievers-bm25 llama-index-llms-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.8 MB/s eta 0:00:00


In [2]:
# 📦 Импорты
from llama_index.core import SimpleDirectoryReader, DocumentSummaryIndex
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.response.notebook_utils import display_source_node
import os, getpass

In [3]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")


Вставь OpenAI API ключ: ··········


In [4]:
# Загрузка документов
documents = SimpleDirectoryReader("faq_data").load_data()

# Разделение документов на узлы
splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)

# Инициализация BM25Retriever
retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=3  # Количество возвращаемых результатов
)

# Выполнение запроса
query = "Какие фирменные шрифты?"
retrieved_nodes = bm25_retriever.retrieve(query)

# Вывод результатов
for node in retrieved_nodes:
    print(node.text)


DEBUG:bm25s:Building index from IDs objects
DEBUG:bm25s:Building index from IDs objects


8 Дополнительная цветовая гамма  
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
� 12
1.9 Фирменные шрифты  
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 �
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
� 13
1.10 Нерекомендуемое использование логотипа  
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
� 14
1.11 Стилеобразующий элемент  
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
� 15
1.12 Варианты использования � 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
�
 
� 16
1.
1.9 Фирменные шрифты
PHENOMENA
Яркий и необычный шрифт. 
Используйте его для выде-
ления заголовков и важных 
сообщений. 
NOTO SANS
Спокойный шрифт 
с большим количеством 
начертаний. Отлично подхо-
дит для текстовых блоков.
Только живая 
МОЖЕТ СДЕЛАТЬ
/uni0441/uni043F/uni043E/uni0441/uni043E/uni043

In [2]:
!pip install -q langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.5 MB/s eta 0:00:00


In [5]:
from langgraph.graph import StateGraph
from typing import TypedDict, List

class SupportState(TypedDict):
    user_type: str
    user_question: str
    tasks: List[str]
    feedback: str

# Инициализация графа
graph = StateGraph(SupportState)

# Определение узлов
def detect_user(state: SupportState) -> SupportState:
    if state["user_type"] == "new":
        state["tasks"] = ["send_welcome", "faq_reply", "log", "ask_feedback"]
    else:
        state["tasks"] = ["handover_to_manager"]
    return state

def task_worker(state: SupportState) -> SupportState:
    task = state["tasks"].pop(0)
    if task == "send_welcome":
        print("👋 Добро пожаловать!")
    elif task == "faq_reply":
        # Предполагается, что query_engine определён ранее
        answer = query_engine.query(state["user_question"])
        print(f"🤖 {answer.response}")
    elif task == "log":
        print("📝 Ответ залогирован.")
    elif task == "ask_feedback":
        state["feedback"] = input("Оцените ответ (1-5): ")
        if int(state["feedback"]) < 3:
            print("📡 Перевожу вас к менеджеру.")
    elif task == "handover_to_manager":
        print("🔄 Менеджер подключится скоро.")
    return state

# Добавление узлов в граф
graph.add_node("detect_user", detect_user)
graph.add_node("task_worker", task_worker)

# Определение точек входа и выхода
graph.set_entry_point("detect_user")
graph.set_finish_point("task_worker")

# Добавление ребра между узлами
graph.add_edge("detect_user", "task_worker")

# Компиляция графа
compiled = graph.compile()


In [6]:
# 🚀 Запуск
compiled.invoke({
    "user_type": "new",
    "user_question": "Как оплатить подписку?"
})

👋 Добро пожаловать!


{'user_type': 'new',
 'user_question': 'Как оплатить подписку?',
 'tasks': ['faq_reply', 'log', 'ask_feedback']}

🔁 Обработка нескольких задач
Если необходимо, чтобы task_worker обрабатывал все задачи из списка tasks, можно добавить условные переходы:

In [ ]:
def has_more_tasks(state: SupportState) -> str:
    return "task_worker" if state["tasks"] else "END"

graph.add_conditional_edges("task_worker", has_more_tasks)


Это обеспечит повторный вызов task_worker, пока в списке tasks остаются задачи.



In [ ]:
# 📥 JSON-фрагмент для n8n
n8n_json = '''[
  {
    "parameters": {},
    "id": "Webhook_Trigger",
    "name": "Webhook",
    "type": "n8n-nodes-base.webhook",
    "typeVersion": 1,
    "position": [
      300,
      300
    ],
    "webhookId": "salebot-new-client",
    "path": "salebot-new-client",
    "method": "POST"
  },
  {
    "parameters": {
      "conditions": {
        "string": [
          {
            "value1": "={{$json[\"user_type\"]}}",
            "operation": "equal",
            "value2": "new"
          }
        ]
      }
    },
    "id": "IF_Check",
    "name": "IF New Client",
    "type": "n8n-nodes-base.if",
    "typeVersion": 1,
    "position": [
      500,
      300
    ]
  },
  {
    "parameters": {
      "prompt": "\u041e\u0442\u0432\u0435\u0442\u044c \u043d\u0430 \u0432\u043e\u043f\u0440\u043e\u0441 \u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u0442\u0435\u043b\u044f, \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u0443\u044f \u0437\u043d\u0430\u043d\u0438\u044f \u0438\u0437 FAQ: {{$json[\"question\"]}}",
      "model": "gpt-3.5-turbo"
    },
    "id": "OpenAI_FAQ",
    "name": "OpenAI",
    "type": "n8n-nodes-base.openai",
    "typeVersion": 1,
    "position": [
      700,
      200
    ]
  }
]'''